# 5th Homework
## András Simon
---

Install the optimization packages

In [1]:
!pip3 install hyperas
!pip3 install hyperopt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import files
import keras
from keras.layers import Layer, Conv2D, Conv2DTranspose, UpSampling2D, MaxPool2D, Flatten, BatchNormalization
from keras.layers import Input, Dense, Dropout, Activation, Add, Concatenate
from keras.datasets import cifar10
from keras import regularizers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
import keras.backend as K
from keras.objectives import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import hyperas
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using the code in the practice Lecture with the CIFAR-10 data

In [3]:
def data():
  # Load the data
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  
  num_classes = 10
  
  # One-hot encoding
  y_train = keras.utils.to_categorical(y_train, 10)
  y_test = keras.utils.to_categorical(y_test, 10)
  
  # Set data type
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  # Normalize
  x_train /= 225  
  x_test /= 225
  
  return x_train, y_train, x_test, y_test

The model is a CNN model

In [4]:
def create_model(x_train, y_train, x_test, y_test):

    class Swish(Layer):
        def __init__(self, beta, **kwargs):
            super(Swish, self).__init__(**kwargs)
            self.beta = K.cast_to_floatx(beta)

        def call(self, inputs):
            return K.sigmoid(self.beta * inputs) * inputs

        def get_config(self):
            config = {'beta': float(self.beta)}
            base_config = super(Swish, self).get_config()
            return dict(list(base_config.items()) + list(config.items()))

        def compute_output_shape(self, input_shape):
            return input_shape
    
    # The hyperparameters of the CNN models to optimize
    channels_1 = {{choice([64, 128, 256, 512])}}
    kernel_size_1 = {{choice([4, 8, 16])}}
    channels_2 = {{choice([64, 128, 256, 512])}}
    kernel_size_2 = {{choice([8, 16])}}
    dense = {{choice([128, 256, 512])}}
    dropout = {{uniform(0, 0.5)}}
    act = {{choice(['relu', 'leakyrelu', 'swish'])}}
    optim = {{choice(['rmsprop', 'adam', 'sgd'])}}
    n_batch = {{choice([64, 128, 256])}}
    print('Hyperparameters of the model: ', channels_1, kernel_size_1, channels_2, kernel_size_2, dense, dropout, act, optim, n_batch)
    
    if act == 'relu':
        activation = keras.layers.ReLU()
    elif act == 'leakyrelu':
        activation = keras.layers.LeakyReLU()
    elif act == 'swish':
        activation = Swish(beta=0.3)
    
    model = Sequential()
    model.add(Conv2D(channels_1, kernel_size_1, padding="same"))
    model.add(activation)
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))

    model.add(Conv2D(channels_2, kernel_size_2, padding="same"))
    model.add(activation)
    model.add(BatchNormalization())
    model.add(MaxPool2D(2))

    model.add(Flatten())
    model.add(Dense(dense))
    model.add(activation)
    model.add(Dropout(dropout))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(optimizer=optim,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    callbacks = [EarlyStopping(monitor='val_acc', patience=3, verbose=0)]
    
    result = model.fit(x_train, y_train,
              batch_size=n_batch,
              epochs=100,
              verbose=2,
              validation_data=(x_test, y_test),
              callbacks=callbacks,
              shuffle=True)

    
    best_val_acc = np.amax(result.history['val_acc']) 
    print('best val_acc:', best_val_acc)
    
    with open('hyperas-fashionmnist-log.csv', 'a') as csv_file:
      csv_file.write(str(n_layer1) + ';')
      csv_file.write(str(n_layer2) + ';')
      csv_file.write(str(dropout_1) + ';')
      csv_file.write(str(dropout_2) + ';')
      csv_file.write(str(act) + ';')
      csv_file.write(str(optim) + ';')
      csv_file.write(str(n_batch) + ';')
      csv_file.write(str(best_val_acc) + '\n')

    return {'loss': -best_val_acc, 'status': STATUS_OK, 'model': model}

Log file

In [5]:
with open('hyperas-cifar10-log.csv', 'w') as csv_file:
  csv_file.write('n_layer1' + ';')
  csv_file.write('n_layer2' + ';')
  csv_file.write('dropout_1' + ';')
  csv_file.write('dropout_2' + ';')
  csv_file.write('act' + ';')
  csv_file.write('optim' + ';')
  csv_file.write('n_batch' + ';')
  csv_file.write('best_val_acc' + '\n')

Upload the notebook itself to Google Colab for the optim.minimize function. 

In [ ]:
src = list(files.upload().values())[0]

In [ ]:
!ls

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          notebook_name='5th_Homework',
                                          trials=Trials())